# PANEL'S BODY RECONSTRUCTION
## finds rectangular shapes
## understand which ones could be a box
## indentifies panel's body



**var**: search for contours in both unmodified and masked image, then try to match contours present in both



**var**: selection wrt attended dimension computed by panel's distance (what's panel dist though?)

**var2**: contours sharing an edge: body's one should share up to 2 lines each


**var**: vanishing point, perspective ratios,

## background remotion
eliminates lots of potential contours, decreases roi

0) prefiltering (enhance features / noise remotion)

1) background catch

2) thresholding by color

999) postfiltering (remotion of artifacts)

## contours detection
detects contourse and straight lines

0) prefiltering

1) edges detection with canny's

2) contours detection

999) postfiltering

## shape recognition
reconstructs the panel shape by looking for rectangular shapes with common edges and specific orientation

0) prefiltering

1) lines detection with hough

2) lines' intersection detection

3) polynomial shapes approximation

4) polynomial shapes testing

5) shapes orientation and body reconstruction



In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt


# BACKGROUND REMOTION

# CONTOURS DETECTION

# SHAPE RECOGNITION

https://docs.opencv.org/4.2.0/

In [ ]:
def mythreshold(img2thr,thrmethod, thrparam):
    
     #adaptive param??
    
    if thrmethod=='canny':
        #requires 3 param , eg: thrparam=(120,255,0)
        ## also thrparam[1]=2*thrparam[0]
        img2thr_grey=cv2.cvtColor(img2thr, cv2.COLOR_RGB2GRAY)
        thrdimg = cv2.Canny(img2thr_grey, thrparam[0], thrparam[1], thrparam[2])
    elif thrmethod=='simple':
        #requires 2 param , eg: thrparam=(127,255)
        img2thr_grey=cv2.cvtColor(img2thr, cv2.COLOR_RGB2GRAY)
        _,thrdimg = cv2.threshold(img2thr_grey,thrparam[0], thrparam[1],cv2.THRESH_BINARY)
        #also cv2.THRESH_BINARY_INV , cv2.THRESH_TRUNC , cv2.THRESH_TOZERO , cv2.THRESH_TOZERO_INV
    elif thrmethod=='otsu':
        #requires 2 param , eg: thrparam=(0,255)
        img2thr_grey=cv2.cvtColor(img2thr, cv2.COLOR_RGB2GRAY)
        _,thrdimg = cv2.threshold(img2thr_grey,thrparam[0], thrparam[1],
                                           cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    elif thrmethod=='adaptive':
        # requires 3 param , eg: thrparam=(255,11,2)
        img2thr_grey=cv2.cvtColor(img2thr, cv2.COLOR_RGB2GRAY)
        thrdimg=cv2.adaptiveThreshold(img2thr_grey,thrparam[0], cv2.ADAPTIVE_THRESH_MEAN_C, 
                                      cv2.THRESH_BINARY,thrparam[1], thrparam[2])
        #also cv2.ADAPTIVE_THRESH_GAUSSIAN_C , same parameters
    elif thrmethod=='killcolor':
        #requires 2 param (vectorial) , eg: thrcol_low=(50,50,50) , thrcol_high=(255,255,255)
        thrcol_low=(thrparam[0],thrparam[1],thrparam[2])
        thrcol_high=(thrparam[3],thrparam[4],thrparam[5]) #color window
        thrimg_hsv=cv2.cvtColor(img2thr, cv2.COLOR_BGR2HSV) #hsv 
        thrmask = cv2.bitwise_not(cv2.inRange(thrimg_hsv, thrcol_low,thrcol_high))
        thrdimg = thrmask# for the mask #cv2.bitwise_and(img2thr,img2thr, mask=thrmask) #for the masked image
    elif thrmethod=='letcolor':
        #requires 2 param (vectorial) ,
        thrcol_low=(thrparam[0],thrparam[1],thrparam[2])
        thrcol_high=(thrparam[3],thrparam[4],thrparam[5])
        thrimg_hsv=cv2.cvtColor(img2thr, cv2.COLOR_BGR2HSV)
        thrmask = cv2.inRange(thrimg_hsv, thrcol_low,thrcol_high)
        thrdimg = thrmask#cv2.bitwise_and(img2thr,img2thr, mask=thrmask)
    #else: print('not binarized!')
    return thrdimg

## switch, elif, dictionaris: which one to use?
#https://www.pydanny.com/why-doesnt-python-have-switch-case.html
#https://stackoverflow.com/questions/60208/replacements-for-switch-statement-in-python
## more efficient way to pass many parameters
#
def myfilter(fltgimg,fltmeth,fltparam):
    
    #adaptive kernel ??
    
    if fltmeth=='gaussian':
        #3param eg: fltparam=(21,21,0)
        filtdimg=cv2.GaussianBlur(fltgimg, (fltparam[0],fltparam[1]),fltparam[2])
    elif fltmeth=='bilateral':
        #3param eg: fltparam=(21,51,51)
        filtdimg=cv2.bilateralFilter(fltgimg,fltparam[0],fltparam[1],fltparam[2])
    #elif fltmeth=='mean':
        #
    elif fltmeth=='median':
        #1param eg: fltparam=(21)
        filtdimg=cv2.medianBlur(img, fltparam[0])
    #else: print('not filtered')
    return filtdimg

#https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html
#https://docs.opencv.org/master/d4/d13/tutorial_py_filtering.html
#https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html

def mymorph(mrpimg,mrpmeth,mrpparam):
# requires 4 param ie: mrpparam=(kern_sizex,kern_sizey,numbiter_erode, numbiter_dilate)    
    #adaptive kernel??
    
    ##also mrpkernel=np.ones((mrpparam[0], mrpparam[1]),np.uint8)
    #if mrpmeth!=0:
        #if mrphparam[0]!=0
    mrpkernel = cv2.getStructuringElement( cv2.MORPH_RECT, ( mrpparam[0], mrpparam[1] ) )
        #else: morphkernel=cv2.getStructuringElement( cv2.MORPH_RECT, (7,7))
    if mrpmeth=='close':
        mrpdimg = cv2.morphologyEx( mrpimg, cv2.MORPH_CLOSE, mrpkernel )
    elif mrpmeth=='open':
        mrpdimg = cv2.morphologyEx( mrpimg, cv2.MORPH_OPEN, mrpkernel )    
    elif mrpmeth=='erode':
        mrpdimg = cv2.erode(mrpdimg,mrpkernel,iterations = mrpparam[2])
    elif mrpmeth=='dilate':
        mrpdimg = cv2.dilate(mrpdimg,mrpkernel,iterations = mrpparam[3])
    #else: print('not morphed')
        
    return mrpdimg

In [1]:
# VIDEO STREAM

import cv2
import time
import numpy as np

##################
DELAY = 0.02
USE_CAM = 1
IS_FOUND = 0

MORPH = 7
CANNY = 250
##################
# 420x600 oranı 105mmx150mm gerçek boyuttaki kağıt için
_width  = 600.0
_height = 420.0
_margin = 0.0
##################

if USE_CAM: video_capture = cv2.VideoCapture(0)

corners = np.array(
    [
    [[  _margin, _margin]],
    [[ _margin, _height + _margin  ]],
    [[ _width + _margin, _height + _margin  ]],
    [[ _width + _margin, _margin ]],
    ]
)

pts_dst = np.array( corners, np.float32 )

while True :

    if USE_CAM :
        ret, rgb = video_capture.read()
    else :
        ret = 1
        rgb = cv2.imread( "opencv.jpg", 1 )

    if ( ret ):

        gray = cv2.cvtColor( rgb, cv2.COLOR_BGR2GRAY )

        gray = cv2.bilateralFilter( gray, 1, 10, 120 )

        edges  = cv2.Canny( gray, 10, CANNY )

        kernel = cv2.getStructuringElement( cv2.MORPH_RECT, ( MORPH, MORPH ) )

        closed = cv2.morphologyEx( edges, cv2.MORPH_CLOSE, kernel )
        _,contours, h = cv2.findContours( closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE )

        for cont in contours:

            # Küçük alanları pass geç
            if cv2.contourArea( cont ) > 5000 :

                arc_len = cv2.arcLength( cont, True )

                approx = cv2.approxPolyDP( cont, 0.2 * arc_len, True )

                if ( len( approx ) == 4 ):
                    IS_FOUND = 1
                    #M = cv2.moments( cont )
                    #cX = int(M["m10"] / M["m00"])abs
                    #cY = int(M["m01"] / M["m00"])
                    #cv2.putText(rgb, "Center", (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

                    pts_src = np.array( approx, np.float32 )

                    h, status = cv2.findHomography( pts_src, pts_dst )
                    out = cv2.warpPerspective( rgb, h, ( int( _width + _margin * 2 ), int( _height + _margin * 2 ) ) )

                    cv2.drawContours( rgb, [approx], -1, ( 255, 0, 0 ), 2 )

                else : pass

        #cv2.imshow( 'closed', closed )
        #cv2.imshow( 'gray', gray )
        

        if IS_FOUND :
            cv2.namedWindow( 'out', cv2.CV_WINDOW_AUTOSIZE )
            cv2.imshow( 'out', out )

        if cv2.waitKey(27) & 0xFF == ord('q') :
            break

        if cv2.waitKey(99) & 0xFF == ord('c') :
            current = str( time.time() )
            cv2.imwrite( 'ocvi_' + current + '_edges.jpg', edges )
            cv2.imwrite( 'ocvi_' + current + '_gray.jpg', gray )
            cv2.imwrite( 'ocvi_' + current + '_org.jpg', rgb )
            print("Pictures saved")

        time.sleep( DELAY )

    else :
      #  print "Stopped"
        break

if USE_CAM : video_capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 